In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import accuracy_score
import numpy as np
import sys
import re
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, InputLayer


In [2]:
vdata = pd.read_csv("VariantTrainingData.csv")
vdata.head(10)

vdata.loc[vdata["Classified"] == 'VOI', "Classified"] = 0
vdata.loc[vdata["Classified"] == 'VOC', "Classified"] = 1
vdata.loc[vdata["Classified"] == 'VUM', "Classified"] = 2

spikeP = vdata["AA Substitutions"]
lineage = vdata["Lineage"]
clade = vdata["Clade"]
variant = vdata["Classified"]

vdata.head(10)


,Virus name,Accession ID,Collection date,Location,Host,Passage,Specimen,Additional host information,Sequencing technology,Assembly method,Comment,Comment type,Lineage,Clade,AA Substitutions,Variant,Classified
0,hCoV-19/England/CAMC-149B04F/2021,EPI_ISL_1483874,3/30/2021,Europe / United Kingdom / England,Human,Original,NaN,NaN,Illumina NovaSeq,NaN,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
1,hCoV-19/USA/NY-CDC-QDX23179684/2021,EPI_ISL_1491965,3/20/2021,North America / USA / New York,Human,Original,Nasal swab,NaN,Illumina MiSeq,BWA v.7.12,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
2,hCoV-19/USA/CA-CDC-FG-010085/2021,EPI_ISL_1481945,3/13/2021,North America / USA / California,Human,Original,Nasal swab,NaN,Illumina NovaSeq,BWA; iVar 1.3,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
3,hCoV-19/USA/TN-CDC-ASC210017485/2021,EPI_ISL_1491924,3/24/2021,North America / USA / Tennessee,Human,Original,Nasal - Anterior Nares,NaN,Illumina NovaSeq 6000,Dragen COVID Lineage v3.5.1,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
4,hCoV-19/USA/TN-CDC-ASC210016026/2021,EPI_ISL_1513430,3/23/2021,North America / USA / Tennessee,Human,Original,Nasal swab,NaN,Illumina NovaSeq 6000,Dragen COVID Lineage v3.5.1,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
5,hCoV-19/USA/TN-CDC-ASC210016009/2021,EPI_ISL_1513424,3/23/2021,North America / USA / Tennessee,Human,Original,Nasopharyngeal swab,NaN,Illumina NovaSeq 6000,Dragen COVID Lineage v3.5.1,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
6,hCoV-19/USA/AR-CDC-LC0035111/2021,EPI_ISL_1515553,3/23/2021,North America / USA / Arkansas,Human,Original,Nasal swab,NaN,PacBio Sequel II,CLC Genomics,Long stretches of NNNs (6.74% of overall seque...,alert,C.37,GR,"(N_T366I,N_G214C,NSP5_G15S,NSP6_G107del,NSP6_S...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
7,hCoV-19/USA/AZ-CDC-LC0035336/2021,EPI_ISL_1515705,3/26/2021,North America / USA / Arizona,Human,Original,Nasal swab,NaN,PacBio Sequel II,CLC Genomics,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
8,hCoV-19/Canada/QC-1nIUH-5524359940/2021,EPI_ISL_2975479,4/5/2021,North America / Canada / Quebec,Human,Original,NaN,NaN,Illumina_NexteraFlex,iVar 1.3,Gap of 9 nucleotides when compared to the refe...,info,C.37,GR,"(Spike_G75V,NSP6_S106del,N_R203K,Spike_F490S,S...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
9,hCoV-19/USA/NY-MSHSPSP-PV35341/2021,EPI_ISL_1709289,3/10/2021,North America / USA / New York / Queens County,Human,Original,NaN,NaN,Illumina MiSeq,"Minimap2, Pilon",Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0


In [3]:
spikeP

0       (Spike_L249del,Spike_S247del,Spike_G75V,Spike_...
1       (Spike_L249del,Spike_S247del,Spike_G75V,Spike_...
2       (Spike_L249del,Spike_S247del,Spike_G75V,Spike_...
3       (Spike_L249del,Spike_S247del,Spike_G75V,Spike_...
4       (Spike_L249del,Spike_S247del,Spike_G75V,Spike_...
                              ...                        
3995    (Spike_T95I,Spike_E484K,NSP7_S26F,NS3_L106F,NS...
3996    (NSP13_P419S,NSP6_Q160R,Spike_T95I,N_A152G,Spi...
3997    (NSP13_P419S,NSP12_M629I,Spike_T95I,Spike_D950...
3998    (Spike_T95I,NSP1_M85del,Spike_E484K,NSP14_L177...
3999    (Spike_T95I,NSP1_M85del,Spike_E484K,NSP14_L177...
Name: AA Substitutions, Length: 4000, dtype: object

In [4]:
cv=CountVectorizer()

In [5]:
x_train, x_test, y_train, y_test = train_test_split(spikeP, variant, test_size=0.2)

x_traincv = cv.fit_transform(x_train)
x_traincv.toarray()
print(type(x_traincv))
print(cv.get_feature_names_out())
print(y_train.astype('int'))

<class 'scipy.sparse._csr.csr_matrix'>
['e_a36v' 'e_d72g' 'e_d72y' ... 'spike_y495h' 'spike_y505h' 'spike_y660h']
151     0
1540    1
142     0
3591    0
1713    1
       ..
1493    1
60      0
1529    1
2529    2
1225    1
Name: Classified, Length: 3200, dtype: int32


In [6]:
multiNom = MultinomialNB()
multiNom.fit(x_traincv, y_train.astype('int'))

MultinomialNB()

In [7]:
x_testcv = cv.transform(x_test)
print(multiNom.score(x_testcv, y_test.astype('int')))

x_predict = multiNom.predict(x_testcv)
print(accuracy_score(x_predict, y_test.astype('int')))
r2_score(x_predict,y_test.astype('int'))

0.99875
0.99875


0.9965851513845077

In [8]:
GausNB = GaussianNB()
GausNB.fit(x_traincv.toarray(),y_train.astype('int'))

print(GausNB.score(x_testcv.toarray(),y_test.astype('int')))


0.99875


In [9]:
seq_data =  pd.read_csv("Covseqdata.csv")
gene_seq = seq_data["Sequence"]

print(type(seq_data))
print(gene_seq)
print(type(gene_seq))
print(seq_data.Sequence.str.len().max())

<class 'pandas.core.frame.DataFrame'>
0       AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...
1       AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...
2       AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...
3       TTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCA...
4       AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...
                              ...                        
4683    ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...
4684    TAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATC...
4685    ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...
4686    ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...
4687    ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...
Name: Sequence, Length: 4688, dtype: object
<class 'pandas.core.series.Series'>
30299


In [10]:
#pd.options.mode.chained_assignment = None
def str_toarray(seqstr):
    seqstr = seqstr.lower()
    if('z' not in seqstr):
        seqstr = seqstr + 'n'
    seqstr = re.sub('[^acgt]', 'z', seqstr)
    seqstr = np.array(list(seqstr))
    return seqstr
def one_hot_encode(seqstr):
    lbl_encoder = LabelEncoder()
    lbl_encoder.fit(np.array(['a','c','g','t','z']))
    num_encoded = lbl_encoder.transform(seqstr)
    num_encoded = num_encoded.reshape(len(num_encoded),1)
    onehot_encoder = OneHotEncoder(sparse_output = False, dtype=float)
    one_encoded = onehot_encoder.fit_transform(num_encoded)
    if (np.count_nonzero(seqstr=='z') == 1):
        #print("did not find z")
        one_encoded[len(one_encoded) - 1] = [0., 0., 0., 0., 0.]
    return one_encoded
one_hot_encode(str_toarray(gene_seq[0]))
onehot_gene_seq = []
#seq_data.loc[seq_data["Sequence"], "One_hot_seq"] = one_hot_encode(str_toarray())
for count, gene in enumerate(gene_seq):
    onehot_gene_seq.insert(count, one_hot_encode(str_toarray(gene)))
    #seq_data.loc[seq_data["Sequence"], "One_hot_seq"] = one_hot_encode(str_toarray(seq_data["Sequence"]))
seq_data['onehot_encode'] = onehot_gene_seq


In [11]:
seq_data.loc[seq_data["Variant"] == 'VOI', "Classified"] = int(0)
seq_data.loc[seq_data["Variant"] == 'VOC', "Classified"] = int(1)
seq_data.loc[seq_data["Variant"] == 'VUM', "Classified"] = int(2)

 
x_seq_data = seq_data["onehot_encode"]
y_seq_data = seq_data["Classified"] 
print(x_seq_data[1].shape)
seq_data.head()



(29715, 5)


,ID,Sequence,Variant,onehot_encode,Classified
0,hCoV-19/Botswana/R43B69_BHP_2421009581/2021|EP...,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,VOC,"[[1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0....",1.0
1,hCoV-19/Botswana/R43B68_BHP_121142361/2021|EPI...,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,VOC,"[[1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0....",1.0
2,hCoV-19/Botswana/R43B15_BHP_000842375/2021|EPI...,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,VOC,"[[1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0....",1.0
3,hCoV-19/Botswana/R43B33_BHP_AAC25682/2021|EPI_...,TTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCA...,VOC,"[[0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1....",1.0
4,hCoV-19/Botswana/R43B70_BHP_4021000195/2021|EP...,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,VOC,"[[1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0....",1.0


In [12]:
# dum = pd.get_dummies(y_seq_data)
# merged = pd.concat([seq_data, dum], axis='columns')
# #seq_data['Classified']=seq_data[['VOC','VOI','VUM']].values.tolist()
# merged['Classified']=merged[['VOC','VOI','VUM']].values.tolist()
# y_cate_data = merged['Classified']
# print(y_cate_data.shape)
# y_onehot_data = []
# for count, category in enumerate(y_cate_data):
#     y_onehot_data.insert(count,np.array(category))
# merged['Classified'] = y_onehot_data
# y_onehot_seq_data = merged['Classified'].to_numpy()
# print(type(y_onehot_seq_data)) 
# print(y_onehot_seq_data[1].shape)
# y_onehot_seq_data

In [13]:
padd_x_seq = pad_sequences(x_seq_data, padding = "post")
print((padd_x_seq.shape))
print((padd_x_seq[0].shape))
print(padd_x_seq)

(4688, 30300, 5)
(30300, 5)
[[[1 0 0 0 0]
  [0 0 1 0 0]
  [1 0 0 0 0]
  ...
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[1 0 0 0 0]
  [0 0 1 0 0]
  [1 0 0 0 0]
  ...
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[1 0 0 0 0]
  [0 0 1 0 0]
  [1 0 0 0 0]
  ...
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 ...

 [[1 0 0 0 0]
  [0 1 0 0 0]
  [0 0 0 1 0]
  ...
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[1 0 0 0 0]
  [0 1 0 0 0]
  [0 0 0 1 0]
  ...
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[1 0 0 0 0]
  [0 1 0 0 0]
  [0 0 0 1 0]
  ...
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]


In [14]:
#x_train_seq, x_test_seq, y_train_seq, y_test_seq = train_test_split(padd_x_seq, y_onehot_seq_data, test_size=0.3)
x_train_seq, x_test_seq, y_train_seq, y_test_seq = train_test_split(padd_x_seq, y_seq_data, test_size=0.3)
print(x_train_seq)
print(y_train_seq)
print(type(x_train_seq))
print(type(y_train_seq))
print(type(x_train_seq[0]))
print(type(y_train_seq[0]))

[[[1 0 0 0 0]
  [0 0 1 0 0]
  [1 0 0 0 0]
  ...
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[1 0 0 0 0]
  [0 0 1 0 0]
  [1 0 0 0 0]
  ...
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[1 0 0 0 0]
  [0 0 1 0 0]
  [1 0 0 0 0]
  ...
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 ...

 [[1 0 0 0 0]
  [0 1 0 0 0]
  [0 1 0 0 0]
  ...
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 1 0]
  [0 0 0 1 0]
  [0 0 1 0 0]
  ...
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[1 0 0 0 0]
  [0 0 1 0 0]
  [1 0 0 0 0]
  ...
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
3280    0.0
2598    1.0
3779    0.0
2315    1.0
2074    1.0
       ... 
3941    0.0
1679    1.0
1562    1.0
334     1.0
1941    1.0
Name: Classified, Length: 3281, dtype: float64
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'numpy.float64'>


In [15]:
y_train_seq = tf.keras.utils.to_categorical(y_train_seq, num_classes=3)
print(y_train_seq)
print(type(y_train_seq))
print(y_train_seq.shape)

[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
<class 'numpy.ndarray'>
(3281, 3)


In [16]:
# implement CNN 
#y_train_seq = tf.keras.utils.to_categorical(y_train_seq, num_classes=3)
from keras.optimizers import SGD
opt = SGD(lr=0.001)
CNNmodel = tf.keras.models.Sequential()
#add convolutional layer
CNNmodel.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', input_shape = (30300,5,1)))

CNNmodel.add(MaxPooling2D(pool_size=(2,2)))

CNNmodel.add(Flatten())
CNNmodel.add(Dense(225, activation='relu'))

CNNmodel.add(Dropout(0.3))
CNNmodel.add((Dense(3,activation='softmax')))

CNNmodel.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
CNNmodel.fit(x_train_seq, y_train_seq, batch_size = 64, epochs = 5)


C:\Users\jxion\CS271\CS271env\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/5


KeyboardInterrupt: 

In [17]:
#implement Kmers
seq_data['sliced_data'] = seq_data['Sequence'].str.slice(0,25000)
print(seq_data.sliced_data.str.len().max())

25000


In [18]:
def get_kmers(seq, size):
    return [seq[x:x+size].lower() for x in range(len(seq) - size + 1)]
sliced_gene_data = seq_data['sliced_data']
#print(get_kmers(sliced_gene_data[1], 5))
kmer_data = []
for count, gene in enumerate(sliced_gene_data):
    #onehot_gene_seq.insert(count, one_hot_encode(str_toarray(gene)))
    kmer_data.insert(count, (get_kmers((gene), 5)))
print(kmer_data[12])
seq_data['kmers'] = kmer_data


['agatc', 'gatct', 'atctg', 'tctgt', 'ctgtt', 'tgttc', 'gttct', 'ttctc', 'tctct', 'ctcta', 'tctaa', 'ctaaa', 'taaac', 'aaacg', 'aacga', 'acgaa', 'cgaac', 'gaact', 'aactt', 'acttt', 'cttta', 'tttaa', 'ttaaa', 'taaaa', 'aaaat', 'aaatc', 'aatct', 'atctg', 'tctgt', 'ctgtg', 'tgtgt', 'gtgtg', 'tgtgg', 'gtggc', 'tggct', 'ggctg', 'gctgt', 'ctgtc', 'tgtca', 'gtcac', 'tcact', 'cactc', 'actcg', 'ctcgg', 'tcggc', 'cggct', 'ggctg', 'gctgc', 'ctgca', 'tgcat', 'gcatg', 'catgc', 'atgct', 'tgctt', 'gctta', 'cttag', 'ttagt', 'tagtg', 'agtgc', 'gtgca', 'tgcac', 'gcact', 'cactc', 'actca', 'ctcac', 'tcacg', 'cacgc', 'acgca', 'cgcag', 'gcagt', 'cagta', 'agtat', 'gtata', 'tataa', 'ataat', 'taatt', 'aatta', 'attaa', 'ttaat', 'taata', 'aataa', 'ataac', 'taact', 'aacta', 'actaa', 'ctaat', 'taatt', 'aatta', 'attac', 'ttact', 'tactg', 'actgt', 'ctgtc', 'tgtcg', 'gtcgt', 'tcgtt', 'cgttg', 'gttga', 'ttgac', 'tgaca', 'gacag', 'acagg', 'cagga', 'aggac', 'ggaca', 'gacac', 'acacg', 'cacga', 'acgag', 'cgagt', 'gagta', 

In [19]:
import gensim 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# def tag_docs(corpus, y_tag):
#     for i, list_of_kmers in enumerate(corpus):
#         yield gensim.models.doc2vec.TaggedDocument(list_of_kmers,tags=y_tag[i])

def tag_docs(corpus, label_type):
    tagged = []
    for i, v in enumerate(corpus):
        label = label_type +'_' + str(i)
        tagged.append(TaggedDocument(v, [label]))
    return tagged

# x_kmer_data = seq_data.kmers
# y_kmer_data = seq_data.Variant

x_kmer_train, x_kmer_test, y_kmer_train, y_kmer_test = train_test_split(seq_data.kmers,seq_data.Classified.astype(int), test_size = 0.3)


In [20]:
x_kmer_train = tag_docs(x_kmer_train, 'Train')

TaggedDocument(words=['aactt', 'acttt', 'ctttc', 'tttcg', 'ttcga', 'tcgat', 'cgatc', 'gatct', 'atctc', 'tctct', 'ctctt', 'tcttg', 'cttgt', 'ttgta', 'tgtag', 'gtaga', 'tagat', 'agatc', 'gatct', 'atctg', 'tctgt', 'ctgtt', 'tgttc', 'gttct', 'ttctc', 'tctct', 'ctcta', 'tctaa', 'ctaaa', 'taaac', 'aaacg', 'aacga', 'acgaa', 'cgaac', 'gaact', 'aactt', 'acttt', 'cttta', 'tttaa', 'ttaaa', 'taaaa', 'aaaat', 'aaatc', 'aatct', 'atctg', 'tctgt', 'ctgtg', 'tgtgt', 'gtgtg', 'tgtgg', 'gtggc', 'tggct', 'ggctg', 'gctgt', 'ctgtc', 'tgtca', 'gtcac', 'tcact', 'cactc', 'actcg', 'ctcgg', 'tcggc', 'cggct', 'ggctg', 'gctgc', 'ctgca', 'tgcat', 'gcatg', 'catgc', 'atgct', 'tgctt', 'gctta', 'cttag', 'ttagt', 'tagtg', 'agtgc', 'gtgca', 'tgcac', 'gcact', 'cactc', 'actca', 'ctcac', 'tcacg', 'cacgc', 'acgca', 'cgcag', 'gcagt', 'cagta', 'agtat', 'gtata', 'tataa', 'ataat', 'taatt', 'aatta', 'attaa', 'ttaat', 'taata', 'aataa', 'ataac', 'taact', 'aacta', 'actaa', 'ctaat', 'taatt', 'aatta', 'attac', 'ttact', 'tactg', 'actgt

In [21]:
x_kmer_train[100]

TaggedDocument(words=['aactt', 'acttt', 'ctttc', 'tttcg', 'ttcga', 'tcgat', 'cgatc', 'gatct', 'atctc', 'tctct', 'ctctt', 'tcttg', 'cttgt', 'ttgta', 'tgtag', 'gtaga', 'tagat', 'agatc', 'gatct', 'atctg', 'tctgt', 'ctgtt', 'tgttc', 'gttct', 'ttctc', 'tctct', 'ctcta', 'tctaa', 'ctaaa', 'taaac', 'aaacg', 'aacga', 'acgaa', 'cgaac', 'gaact', 'aactt', 'acttt', 'cttta', 'tttaa', 'ttaaa', 'taaaa', 'aaaat', 'aaatc', 'aatct', 'atctg', 'tctgt', 'ctgtg', 'tgtgt', 'gtgtg', 'tgtgg', 'gtggc', 'tggct', 'ggctg', 'gctgt', 'ctgtc', 'tgtca', 'gtcac', 'tcact', 'cactc', 'actcg', 'ctcgg', 'tcggc', 'cggct', 'ggctg', 'gctgc', 'ctgca', 'tgcat', 'gcatg', 'catgc', 'atgct', 'tgctt', 'gctta', 'cttag', 'ttagt', 'tagtg', 'agtgc', 'gtgca', 'tgcac', 'gcact', 'cactc', 'actca', 'ctcac', 'tcacg', 'cacgc', 'acgca', 'cgcag', 'gcagt', 'cagta', 'agtat', 'gtata', 'tataa', 'ataat', 'taatt', 'aatta', 'attaa', 'ttaat', 'taata', 'aataa', 'ataac', 'taact', 'aacta', 'actaa', 'ctaat', 'taatt', 'aatta', 'attac', 'ttact', 'tactg', 'actgt

In [22]:
x_kmer_test = tag_docs(x_kmer_test, 'Test')

In [24]:
all_data = x_kmer_train + x_kmer_test

In [25]:
from sklearn import utils

dbow_model = Doc2Vec(dm = 0, vector_size = 300, negative =5, min_count =1, alpha =0.065, min_alpha = 0.065)
dbow_model.build_vocab([x for x in all_data])

for epoch in range(30):
    dbow_model.train(utils.shuffle([x for x in all_data]), total_examples = len(all_data), epochs = 1)
    dbow_model.alpha -= 0.002
    dbow_model.min_alpha = dbow_model.alpha

In [30]:
def get_vectors(model, corpus_size, vector_size,vector_type):
    vectors = np.zeros((corpus_size, vector_size))
    for i in range(0,corpus_size):
        prefix = vector_type + '_' +str(i)
        vectors[i]=model.dv[prefix]
    return vectors


In [39]:
train_vectors_dbow = get_vectors(dbow_model, len(x_kmer_train), 300, 'Train')
test_vectors_dbow = get_vectors(dbow_model,len(x_kmer_test),300,'Test')

In [40]:
#put results here
print(train_vectors_dbow[:5])
print(test_vectors_dbow[:5])

[[ 0.11640543 -0.04468325  0.14195231 ... -0.15525289 -0.01478111
  -0.06358602]
 [-0.15400171 -0.0118145   0.16654082 ... -0.10591755  0.03696035
  -0.14614245]
 [-0.0492333  -0.09768508  0.19870275 ... -0.06636102  0.0899477
  -0.17493288]
 [-0.08728651 -0.05844579  0.12952082 ... -0.0586354   0.00123227
  -0.13664037]
 [-0.02674779 -0.04243505  0.08479148 ... -0.11891878  0.03274861
  -0.13445958]]
[[-0.17493944 -0.1849225   0.11103364 ... -0.07075363 -0.05747777
  -0.14756419]
 [-0.02927737 -0.09730896  0.0533241  ... -0.05992229  0.05212145
  -0.09508584]
 [-0.06177718 -0.0939204   0.06768829 ... -0.07999918 -0.04601549
  -0.17631423]
 [ 0.08566171 -0.03874339  0.10190108 ... -0.15798374 -0.08984454
  -0.04290126]
 [-0.0834106  -0.1868526   0.09043535 ... -0.0924986   0.0207395
  -0.09777176]]


In [47]:
#logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
#can change parameters of logistic regression to improve accuracy score. (try C=1e5 as a parameter in "LogisticRegression")
log_reg_model = LogisticRegression(C=1e5)
log_reg_model = log_reg_model.fit(train_vectors_dbow, y_kmer_train)
model_predict = log_reg_model.predict(test_vectors_dbow)
print('Accuracy %s' % accuracy_score(model_predict,y_kmer_test))
print(classification_report(y_kmer_test,model_predict))

Accuracy 0.923951670220327
              precision    recall  f1-score   support

           0       0.90      0.93      0.92       456
           1       0.94      0.93      0.93       776
           2       0.91      0.89      0.90       175

    accuracy                           0.92      1407
   macro avg       0.92      0.92      0.92      1407
weighted avg       0.92      0.92      0.92      1407



C:\Users\jxion\CS271\CS271env\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [48]:
dbow_model_dm = Doc2Vec(dm = 1, vector_size = 300, negative =5, min_count =1, alpha =0.065, min_alpha = 0.065)
dbow_model_dm.build_vocab([x for x in all_data])

for epoch in range(30):
    dbow_model_dm.train(utils.shuffle([x for x in all_data]), total_examples = len(all_data), epochs = 1)
    dbow_model_dm.alpha -= 0.002
    dbow_model_dm.min_alpha = dbow_model.alpha
    
def get_vectors(model, corpus_size, vector_size,vector_type):
    vectors = np.zeros((corpus_size, vector_size))
    for i in range(0,corpus_size):
        prefix = vector_type + '_' +str(i)
        vectors[i]=model.dv[prefix]
    return vectors

train_vectors_dbow = get_vectors(dbow_model_dm, len(x_kmer_train), 300, 'Train')
test_vectors_dbow = get_vectors(dbow_model_dm,len(x_kmer_test),300,'Test')

#logistic regression
#can change parameters of logistic regression to improve accuracy score. (try C=1e5 as a parameter in "LogisticRegression")
log_reg_model = LogisticRegression(C=1e5)
log_reg_model = log_reg_model.fit(train_vectors_dbow, y_kmer_train)
model_predict = log_reg_model.predict(test_vectors_dbow)
print('Accuracy %s' % accuracy_score(model_predict,y_kmer_test))
print(classification_report(y_kmer_test,model_predict))

Accuracy 0.5323383084577115
              precision    recall  f1-score   support

           0       0.41      0.27      0.32       456
           1       0.58      0.80      0.67       776
           2       0.12      0.02      0.03       175

    accuracy                           0.53      1407
   macro avg       0.37      0.36      0.34      1407
weighted avg       0.47      0.53      0.48      1407



C:\Users\jxion\CS271\CS271env\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
